# TSMixer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import time
from random import randrange
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.7)-5
    val_end = int(n * 0.7)+5
    test_end = n

test_true=df[val_end - 21 : test_end]
test_true=test_true.iloc[21:]

# Test

In [ ]:
random_seeds=[]
for i in range(20):
  random_seeds.append(randrange(129228148))

In [ ]:
class DataLoader:

  def __init__(self, batch_size, seq_len, pred_len):
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.target_slice = slice(0, None)

    self._read_data()

  def _read_data(self):

    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.7)-5
    val_end = int(n * 0.7)+5
    test_end = n

    train_df = df[:train_end]
    val_df = df[train_end - self.seq_len : val_end]
    test_df = df[val_end - self.seq_len : test_end]

    # standardize by training set
    self.scaler = StandardScaler()
    self.scaler.fit(train_df.values)

    def scale_df(df, scaler):
      data = scaler.transform(df.values)
      return pd.DataFrame(data, index=df.index, columns=df.columns)

    self.train_df = scale_df(train_df, self.scaler)
    self.val_df = scale_df(val_df, self.scaler)
    self.test_df = scale_df(test_df, self.scaler)
    self.n_feature = self.train_df.shape[-1]

  def _split_window(self, data):
    inputs = data[:, : self.seq_len, :]
    labels = data[:, self.seq_len :, self.target_slice]

    inputs.set_shape([None, self.seq_len, None])
    labels.set_shape([None, self.pred_len, None])
    return inputs, labels

  def _make_dataset(self, data, shuffle=False):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=(self.seq_len + self.pred_len),
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=self.batch_size,
    )
    ds = ds.map(self._split_window)
    return ds

  def inverse_transform(self, data):
    return self.scaler.inverse_transform(data)

  def get_train(self, shuffle=True):
    return self._make_dataset(self.train_df, shuffle=shuffle)

  def get_val(self):
    return self._make_dataset(self.val_df, shuffle=False)

  def get_test(self):
    return self._make_dataset(self.test_df, shuffle=False)

  def make_forecast(self, model):
    test_data = self.test_df.values
    predictions = []

    for i in range(0, len(test_data) - self.seq_len, self.pred_len):
        input_seq = test_data[i:i + self.seq_len]
        input_seq = input_seq.reshape((1, self.seq_len, -1))

            # Make prediction and reshape output
        pred = model.predict(input_seq)
        pred = pred.reshape(self.pred_len, -1)

        predictions.append(pred)

        # Concatenate all predictions
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
RMSE_test=[]
MAE_test=[]
MAPE_test=[]
QLIKE_test=[]

for i in range(5):
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168 #accounting for early stop
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  predictions = model.predict(test_data)
  scaled_preds = predictions
  scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
  scaled_preds.shape
  preds = data_loader.inverse_transform(scaled_preds)
  RMSE_test.append(np.sqrt(((test_true.iloc[:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAE_test.append(((abs(test_true.iloc[:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAPE_test.append(((abs(test_true.iloc[:]-preds)/test_true.iloc[:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  QLIKE_test.append(((test_true.iloc[:]/preds)-np.log(test_true.iloc[:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))

Epoch 1/168
74/74 [==============================] - 15s 11ms/step - loss: 13.7523 - mae: 2.7297 - mean_absolute_percentage_error: 4380.2671 - mean_squared_logarithmic_error: 0.7850
Epoch 2/168
74/74 [==============================] - 1s 17ms/step - loss: 6.6984 - mae: 1.9537 - mean_absolute_percentage_error: 5378.5762 - mean_squared_logarithmic_error: 0.5117
Epoch 3/168
74/74 [==============================] - 1s 16ms/step - loss: 4.3013 - mae: 1.5606 - mean_absolute_percentage_error: 8353.6631 - mean_squared_logarithmic_error: 0.3813
Epoch 4/168
74/74 [==============================] - 1s 13ms/step - loss: 3.0488 - mae: 1.3103 - mean_absolute_percentage_error: 4121.6802 - mean_squared_logarithmic_error: 0.3030
Epoch 5/168
74/74 [==============================] - 1s 11ms/step - loss: 2.3688 - mae: 1.1463 - mean_absolute_percentage_error: 1571.4614 - mean_squared_logarithmic_error: 0.2503
Epoch 6/168
74/74 [==============================] - 1s 11ms/step - loss: 1.8715 - mae: 1.0150 - m

In [ ]:
for i in range(5):
  i=i+5*1
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  predictions = model.predict(test_data)
  scaled_preds = predictions
  scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
  scaled_preds.shape
  preds = data_loader.inverse_transform(scaled_preds)
  RMSE_test.append(np.sqrt(((test_true.iloc[:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAE_test.append(((abs(test_true.iloc[:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAPE_test.append(((abs(test_true.iloc[:]-preds)/test_true.iloc[:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  QLIKE_test.append(((test_true.iloc[:]/preds)-np.log(test_true.iloc[:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))

Epoch 1/168
74/74 [==============================] - 5s 11ms/step - loss: 13.3351 - mae: 2.7092 - mean_absolute_percentage_error: 7204.2671 - mean_squared_logarithmic_error: 0.7830
Epoch 2/168
74/74 [==============================] - 1s 11ms/step - loss: 6.2160 - mae: 1.8878 - mean_absolute_percentage_error: 2541.7400 - mean_squared_logarithmic_error: 0.4871
Epoch 3/168
74/74 [==============================] - 1s 11ms/step - loss: 3.8602 - mae: 1.4836 - mean_absolute_percentage_error: 2096.3240 - mean_squared_logarithmic_error: 0.3498
Epoch 4/168
74/74 [==============================] - 1s 11ms/step - loss: 2.6764 - mae: 1.2285 - mean_absolute_percentage_error: 2105.4900 - mean_squared_logarithmic_error: 0.2689
Epoch 5/168
74/74 [==============================] - 1s 11ms/step - loss: 2.0166 - mae: 1.0621 - mean_absolute_percentage_error: 2191.5642 - mean_squared_logarithmic_error: 0.2207
Epoch 6/168
74/74 [==============================] - 1s 13ms/step - loss: 1.5889 - mae: 0.9376 - me

In [ ]:
for i in range(5):
  i=i+5*2
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  predictions = model.predict(test_data)
  scaled_preds = predictions
  scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
  scaled_preds.shape
  preds = data_loader.inverse_transform(scaled_preds)
  RMSE_test.append(np.sqrt(((test_true.iloc[:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAE_test.append(((abs(test_true.iloc[:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAPE_test.append(((abs(test_true.iloc[:]-preds)/test_true.iloc[:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  QLIKE_test.append(((test_true.iloc[:]/preds)-np.log(test_true.iloc[:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))

Epoch 1/168
74/74 [==============================] - 5s 11ms/step - loss: 12.2809 - mae: 2.5939 - mean_absolute_percentage_error: 6385.3926 - mean_squared_logarithmic_error: 0.6685
Epoch 2/168
74/74 [==============================] - 1s 11ms/step - loss: 5.6196 - mae: 1.7957 - mean_absolute_percentage_error: 2371.3784 - mean_squared_logarithmic_error: 0.4352
Epoch 3/168
74/74 [==============================] - 1s 11ms/step - loss: 3.5137 - mae: 1.4181 - mean_absolute_percentage_error: 7637.8062 - mean_squared_logarithmic_error: 0.3261
Epoch 4/168
74/74 [==============================] - 1s 11ms/step - loss: 2.5040 - mae: 1.1897 - mean_absolute_percentage_error: 3719.4966 - mean_squared_logarithmic_error: 0.2625
Epoch 5/168
74/74 [==============================] - 1s 11ms/step - loss: 1.8735 - mae: 1.0239 - mean_absolute_percentage_error: 2818.0442 - mean_squared_logarithmic_error: 0.2138
Epoch 6/168
74/74 [==============================] - 1s 11ms/step - loss: 1.4572 - mae: 0.8974 - me

In [ ]:
for i in range(5):
  i=i+5*3
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  predictions = model.predict(test_data)
  scaled_preds = predictions
  scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
  scaled_preds.shape
  preds = data_loader.inverse_transform(scaled_preds)
  RMSE_test.append(np.sqrt(((test_true.iloc[:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAE_test.append(((abs(test_true.iloc[:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  MAPE_test.append(((abs(test_true.iloc[:]-preds)/test_true.iloc[:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))))
  QLIKE_test.append(((test_true.iloc[:]/preds)-np.log(test_true.iloc[:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))

Epoch 1/168
74/74 [==============================] - 5s 11ms/step - loss: 18.1216 - mae: 3.0677 - mean_absolute_percentage_error: 7791.2129 - mean_squared_logarithmic_error: 0.7837
Epoch 2/168
74/74 [==============================] - 1s 11ms/step - loss: 8.6522 - mae: 2.2072 - mean_absolute_percentage_error: 6952.7754 - mean_squared_logarithmic_error: 0.5510
Epoch 3/168
74/74 [==============================] - 1s 11ms/step - loss: 5.5905 - mae: 1.7772 - mean_absolute_percentage_error: 2731.2378 - mean_squared_logarithmic_error: 0.4301
Epoch 4/168
74/74 [==============================] - 1s 11ms/step - loss: 4.1106 - mae: 1.5210 - mean_absolute_percentage_error: 4271.2788 - mean_squared_logarithmic_error: 0.3554
Epoch 5/168
74/74 [==============================] - 1s 11ms/step - loss: 3.1245 - mae: 1.3216 - mean_absolute_percentage_error: 5126.0845 - mean_squared_logarithmic_error: 0.2987
Epoch 6/168
74/74 [==============================] - 1s 11ms/step - loss: 2.4680 - mae: 1.1715 - me

In [ ]:
RMSE=RMSE_test
MAE=MAE_test
MAPE=MAPE_test
QLIKE=QLIKE_test

In [ ]:
print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("TSMixer forecasts.xlsx")

RMSE mean: 0.003694624335467369 | QLIKE mean: 0.030385667273665883 | MAE mean: 0.00240569409646647 | MAPE mean: 0.1927214728979697
RMSE std: 1.5329657324195686e-05 | QLIKE std: 8.078238870815387e-05 | MAE std: 1.5640797701226555e-05 | MAPE std: 0.0023212298238755673
RMSE: AndersonResult(statistic=0.47726779990947676, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.003694624335467369, scale=1.57278965595084e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.41306010842542307, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.030385667273665883, scale=8.288098204429406e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.15456539048020446